# Measurement

In [ ]:
from sciopy import available_serial_ports, connect_COM_port
from sciopy.sciopy_dataclasses import ScioSpecMeasurementSetup
import time
from src.classes import BallObjectProperties, HitBox, TankProperties32x2, Ender5Stat
from src.functions import (
    compute_hitbox,
    create_meas_coordinates,
    create_measurement_directory,
    print_coordinates_props,
    save_parameters_to_json_file,
)
from src.visualization import plot_meas_coords, plot_meas_coords_wball

from sciopy import (
    StartStopMeasurement_usb_hs,
    SystemMessageCallback_usb_hs,
    connect_COM_port_usb_hs,
    set_measurement_config_usb_hs,
    available_serial_ports,
    del_hex_in_list,
    reshape_full_message_in_bursts,
    split_bursts_in_frames,
)


from src.ender5 import init_ender5, x_y_z_home, move_ender_to_coordinate

## Define the parameters

In [ ]:
# init tank
tank = TankProperties32x2()
# init ball
ball = BallObjectProperties(x=0, y=0, z=0, r=30, material=None)
# init hitbox
hitbox = compute_hitbox(tank, ball, safety_tolerance=0)
# set sciospec measurement properties
ssms = ScioSpecMeasurementSetup(
    burst_count=1,  # The number of measurements to be performed.
    total_meas_num=10,  # Repetitions of burst count
    n_el=64,  # Set 16, 32, 48 or 64 and do not forget to change the channel_group.
    channel_group=[
        1,
        2,
        3,
        4,
    ],  # Use [1] for n_el=16, [1,2] for n_el=32, [1,2,3] for n_el=48, and [1,2,3,4] for n_el=64
    exc_freq=20_000,  # 10,000Hz = 10kHz
    framerate=5,  # Measurements per second
    amplitude=0.01,  # 0.01A = 10mA (maximum)
    inj_skip=8,  # injection electrode skip
    gain=1,
    adc_range=1,  # +/- 1V
    notes="None",  # add measurement information
    configured=False,  # will be set to true after sending config to device
)

print(tank)
print(ball)
print(hitbox)
print(ssms)

In [ ]:
coordinates = create_meas_coordinates(hitbox, x_pts=3, y_pts=3, z_pts=3)
print_coordinates_props(coordinates)

In [ ]:
plot_meas_coords(tank=tank, meas_coords=coordinates, p_select=0)

In [ ]:
s_path, f_name = create_measurement_directory()
save_parameters_to_json_file(s_path, f_name, ssms, tank, ball, hitbox)

## Connect Devices

- Create 3D objects
- Init Ender 5
- Centering object depending on size
- Init Sciospec
- Send config to sciospec
- Start Measurement

In [ ]:
COM_Ender = connect_COM_port(port="COM4", baudrate=115200)
COM_Sciospec = connect_COM_port_usb_hs()

In [ ]:
enderstat = Ender5Stat(
    abs_x_pos=None,
    abs_y_pos=None,
    abs_z_pos=None,
    tank_architecture=TankProperties32x2(),
    motion_speed=180,
)

In [ ]:
# init axis -> RUN ONLY IF NO OBJECT IS IN THE MOUNTING CONSTRUCTION
init_ender5(COM_Ender, enderstat)

In [ ]:
# move to home position
x_y_z_home(COM_Ender, enderstat, print_msg=True)

## Set Sciospec Config

In [ ]:
set_measurement_config_usb_hs(COM_Sciospec, ssms)

In [ ]:
# Read out system callback
SystemMessageCallback_usb_hs(COM_Sciospec, prnt_msg=True)

# Measure

In [ ]:
import serial
import numpy as np
from src.ender5 import move_to_absolute_x_y_z
import time


def move_ender_to_coordinate(
    ser: serial.serialwin32.Serial,
    coordinate: np.ndarray,
    enderstat: Ender5Stat,
    print_msg: bool = False,
) -> None:
    """
    Move to the P(x,y,z) position of a np.array([x,y,z]).

    Parameters
    ----------
    ser : serial.serialwin32.Serial
        serial connection
    coordinate : np.ndarray
        array with [x,y,z] coordinate
    enderstat : Ender5Stat
        ender 5 dataclass
    print_msg : bool, optional
        print log, by default False
    """
    x_y_offset = 180  # x,y center point
    start_point = np.array(
        [
            enderstat.abs_x_pos - x_y_offset,
            enderstat.abs_y_pos - x_y_offset,
            enderstat.abs_z_pos,
        ]
    )
    distance = np.linalg.norm(start_point - coordinate)
    y_ender, x_ender, z_ender = coordinate  # switch x,y for ender koordinate system

    enderstat.abs_x_pos = x_y_offset + x_ender
    enderstat.abs_y_pos = x_y_offset + y_ender
    enderstat.abs_z_pos = z_ender
    move_to_absolute_x_y_z(ser, enderstat, print_msg)
    time.sleep(
        int(np.ceil((distance / enderstat.motion_speed) * 100) + 4)
    )  # 4 seconds tolerance
    if print_msg:
        print(enderstat)

In [ ]:
# move to start position
move_ender_to_coordinate(COM_Ender, coordinates[0], enderstat, print_msg=True)

In [ ]:
x_y_z_home(COM_Ender, enderstat)

In [ ]:
for XYZ in coordinates[:5]:
    print(XYZ)
    move_ender_to_coordinate(COM_Ender, XYZ, enderstat, print_msg=True)
    time.sleep(3)
    print("da")

In [ ]:
# Start and stop single measurement
measurement_data_hex = StartStopMeasurement_usb_hs(COM_Sciospec)
# Delete hex in mesured buffer
measurement_data = del_hex_in_list(measurement_data_hex)

measurement_data = split_bursts_in_frames(
    split_measurement_data, sciospec_measurement_setup
)